<a href="https://colab.research.google.com/github/maggiemcc02/HessianMonitorWork/blob/main/MHessian_TestECont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook is my first attempt at implementing HR's (5.192) Hessian based monitor function

In [ ]:
# mount my google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install firedrake

# hide output
%%capture

try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window

%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [ ]:
# import firedrake tools
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt # firedrake makes use of matplotlib tools
from firedrake.pyplot import tripcolor, tricontour, triplot #firedrake plotting
from IPython.display import display
from mpl_toolkits import mplot3d
import scipy as sci


# The Known Solution

In [ ]:
def known_solution(x, epsilon):


  eps = Constant(epsilon)

  parta = cos((pi/2)*x[0]) - ( exp(-x[0]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )
  partb = 1 - x[1] - ( exp(-x[1]/eps) - exp(-1/eps) ) / ( 1 - exp(-1/eps) )
  u_exact = parta * partb

  return u_exact

# Physical Solver

In [ ]:

# phyiscal solver

def physical_solver(mesh, x, W2, epsilon = 0.1, plots = True):

  # test and trial functions

  u_hat = TrialFunction(W2)
  v_hat = TestFunction(W2)


  # set epsilon as a firedrake constant


  eps = Constant(epsilon)



  # The jacobian matrix of the mesh solution x

  J_F = grad(x) # jacobian matrix


  #pull components out
  x_xi = J_F[0,0]
  x_eta = J_F[0,1]
  y_xi = J_F[1,0]
  y_eta = J_F[1,1]

  # determinant
  det_J = x_xi * y_eta - x_eta * y_xi


  # inverse of the jacobian


  J_F_inv = inv(J_F)
  J_F_inv_T = transpose(J_F_inv)


  # compute the rhs of the pde

  a1 = (1/det_J) * as_vector([y_eta, -x_eta])
  a2 = (1/det_J) * as_vector([-y_xi, x_xi])

  grad_xi_sol = grad( known_solution(x, epsilon) )
  grad_x_sol = dot(J_F_inv_T, grad_xi_sol)

  val1 = inner( det_J * a1 , grad_x_sol )
  val2 = inner( det_J * a2 , grad_x_sol )

  div_term = (1/det_J) * div( as_vector([val1, val2]) )

  f_hat = -eps**2 * div_term + known_solution(x, epsilon)



  # compute grad_(x) of u (HR 3.8)

  grad_x_u = dot(J_F_inv_T, grad(u_hat))


  # compute grad_(x) of v (HR 3.8)

  grad_x_v = dot(J_F_inv_T, grad(v_hat))


  # The bilinear lhs
  # based on weak forms in Gockenbach and a change of variables

  a = ( (eps**2) * inner( grad_x_u, grad_x_v) + inner(u_hat, v_hat) ) * det_J * dx

  # The linear rhs

  L = (inner( f_hat, v_hat ) * det_J) * dx



  # define the BC's

  bcs = DirichletBC(W2, known_solution(x, epsilon), "on_boundary")


  # solve the problem

  sol = Function(W2)


  solve(a == L, sol, bcs = bcs)


  # exact solution

  exact_sol = known_solution(x, epsilon)


  # L2 error

  phys_error_1 = norm( sqrt(det_J) * (exact_sol - sol) )

  phys_error_2 = norm( sqrt(det_J) * dot(J_F_inv_T ,grad(exact_sol - sol)))




  # return the solution and errors


  return sol, phys_error_1, phys_error_2, J_F, det_J

# Mesh Density Function (Hessian Based)

In [ ]:
# |H(u)| code

class AbsoluteValueH(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""



      H = self.ufl_operands[0] # my matrix : as_matrix([[func1, func2], [func3, func4]]) where funci is in DG0


      HSpace = FunctionSpace(mesh, "DG", 0)

      # extract data

      H00 = assemble(Function(HSpace).interpolate(H[0,0]))
      H01 = assemble(Function(HSpace).interpolate(H[0,1]))
      H10 = assemble(Function(HSpace).interpolate(H[1,0]))
      H11 = assemble(Function(HSpace).interpolate(H[1,1]))


      print(H00)
      print(type(H00))
      print()

      # pointwise array

      Hm = np.array([[H00, H01], [H10, H11]])

      print(Hm)
      print()

      abs_H = sci.linalg.sqrtm(Hm)


      # create firedrake result

      H00_result = Function(HSpace).interpolate(abs_H[0, 0])
      H01_result = Function(HSpace).interpolate(abs_H[0, 1])
      H10_result = Function(HSpace).interpolate(abs_H[1, 0])
      H11_result = Function(HSpace).interpolate(abs_H[1, 1])


      abs_H_result = as_matrix([ [ H00_result, H01_result], [ H10_result, H11_result ] ])


      return abs_H_result



In [ ]:
# spectral norm code


class NormB(AbstractExternalOperator):

  def __init__(self, *operands, function_space, **kwargs):
      AbstractExternalOperator.__init__(self, *operands, function_space=function_space, **kwargs)

  @assemble_method(0, (0,))
  def assemble_N(self, *args, **kwargs):
      """Evaluate N"""



      B = self.ufl_operands[0] # my matrix : as_matrix([[func1, func2], [func3, func4]]) where funci is in DG0


      HSpace = FunctionSpace(mesh, "DG", 0)

      # extract data

      B00 = assemble(Function(HSpace).interpolate(B[0,0]) * dx)
      B01 = assemble(Function(HSpace).interpolate(B[0,1]) * dx)
      B10 = assemble(Function(HSpace).interpolate(B[1,0]) * dx)
      B11 = assemble(Function(HSpace).interpolate(B[1,1]) * dx)


      # pointwise array

      Bm = np.array([[B00, B01], [B10, B11]])


      eigs = sci.linalg.eigvals(Bm)

      spect_norm = max(abs(eigs[0]), abs(eigs[1]))


      # create firedrake result

      norm_result = Function(HSpace).interpolate(spect_norm)


      return norm_result

In [ ]:
# external operator M

def ext_M(mesh, x, phys_sol, Jm, det_Jm, W2, V2,  d, q, m, epsilon, alpha_hat = 1):


  # Computing the Scalars


  # (4) Compute <u>_{L^P(omega)} = S u^2 |J| dxi

  alpha_t1 = alpha_hat * assemble(  (inner(phys_sol, phys_sol) * det_Jm * dx ) )**(1/2)

  # (5) compute alpha^tilda

  t1 =  (d*q) / ( d + q*(2-m) ) - 1
  t2 = 0
  gamma = max(t1, t2)

  alpha_tilda = 2*(2**(gamma + 1))

  # (6) Compute alpha

  alpha = max( alpha_t1, alpha_tilda )

  if alpha == alpha_t1:
    print("I choose first alpha")
  else:
    print("I choose second alpha")
  print()
  print()

  # powers

  p1 = -1/(d + q*(2-m))
  p2 = (m*q) / (d + q*(2-m))



  # Compute the Hessian

  J_inv = inv(Jm)
  J_inv_T = J_inv.T
  gradu = dot( J_inv_T, grad(phys_sol) )
  ux = gradu[0]
  uy = gradu[1]
  grad2ux = dot( J_inv_T, grad(ux) )
  grad2uy = dot( J_inv_T, grad(uy) )
  u_xx = grad2ux[0]
  u_yy = grad2uy[1]
  u_yx = grad2uy[0]
  u_xy = grad2ux[1]
  Hu = as_matrix([[u_xx, u_xy], [u_xy, u_yy]])

  # compute the product

  Hu_prod = dot(Hu.T, Hu)

  # Hu00 = Function(W2).interpolate(Hu_prod[0,0])
  # Hu01 = Function(W2).interpolate(Hu_prod[0,1])
  # Hu10 = Function(W2).interpolate(Hu_prod[1,0])
  # Hu11 = Function(W2).interpolate(Hu_prod[1,1])

  # external operator

  absH = AbsoluteValueH(Hu_prod, function_space = W2).assemble_N()


  # Create the B matrix

  B = Identity(2) + (1/alpha) * absH

  # compute norm B term

  norm_B = NormB(B, function_space = W2).assemble_N()

  norm_B_term = norm_B ** p2

  # compute detB term

  det_B = det(B)**p1

  # the result

  M = det_B * norm_B_term * B

  M_func1 = Function(W2).interpolate(M[0, 0])
  M_func2 = Function(W2).interpolate(M[0, 1])
  M_func3 = Function(W2).interpolate(M[1, 0])
  M_func4 = Function(W2).interpolate(M[1, 1])


  return M, M_func1, M_func2, M_func3, M_func4


In [ ]:
def Sci_Hessian_M_3(mesh, x, phys_sol, Jm, det_Jm, W2, V2,  d, q, m, epsilon, alpha_hat = 1):

  # p = 2

  print('Computing M with alpha_hat = ', alpha_hat)
  print()



  # compute the scalars


  # (4) Compute <u>_{L^P(omega)} = S u^2 |J| dxi

  alpha_t1 = alpha_hat * assemble(  (inner(phys_sol, phys_sol) * det_Jm * dx ) )**(1/2)

  # (5) compute alpha^tilda

  t1 =  (d*q) / ( d + q*(2-m) ) - 1
  t2 = 0
  gamma = max(t1, t2)

  alpha_tilda = 2*(2**(gamma + 1))

  # (6) Compute alpha

  alpha = max( alpha_t1, alpha_tilda )

  if alpha == alpha_t1:
    print("I choose first alpha")
  else:
    print("I choose second alpha")
  print()
  print()

  # powers

  p1 = -1/(d + q*(2-m))
  p2 = (m*q) / (d + q*(2-m))






  # (1) create the Hessian MatrixH(u)

  J_inv = inv(Jm)
  J_inv_T = J_inv.T
  gradu = dot( J_inv_T, grad(phys_sol) )
  ux = gradu[0]
  uy = gradu[1]
  grad2ux = dot( J_inv_T, grad(ux) )
  grad2uy = dot( J_inv_T, grad(uy) )
  u_xx = grad2ux[0]
  u_yy = grad2uy[1]
  u_yx = grad2uy[0]
  u_xy = grad2ux[1]
  Hu = as_matrix([[u_xx, u_xy], [u_xy, u_yy]])

  # eps = Constant(epsilon)

  # u_xx = (-pi**2*cos(pi*x[0]/2)/4 - exp(-x[0]/eps)/(eps**2*(1 - exp(-1/eps))))*(-x[1] + 1 - (exp(-x[1]/eps) - exp(-1/eps))/(1 - exp(-1/eps)))
  # u_yy = -(cos(pi*x[0]/2) - (exp(-x[0]/eps) - exp(-1/eps))/(1 - exp(-1/eps)))*exp(-x[1]/eps)/(eps**2*(1 - exp(-1/eps)))
  # u_xy = (-1 + exp(-x[1]/eps)/(eps*(1 - exp(-1/eps))))*(-pi*sin(pi*x[0]/2)/2 + exp(-x[0]/eps)/(eps*(1 - exp(-1/eps))))
  # Hu = as_matrix([[u_xx, u_xy], [u_xy, u_yy]])


  # (2) Compute H^TH


  Hu_prod = dot(Hu.T, Hu)

  Hu00 = Function(W2).interpolate(Hu_prod[0,0])
  Hu01 = Function(W2).interpolate(Hu_prod[0,1])
  Hu10 = Function(W2).interpolate(Hu_prod[1,0])
  Hu11 = Function(W2).interpolate(Hu_prod[1,1])


  # place to store each sqrt matrix
  n_comp = len(Function(W2).interpolate(u_xx).dat.data)

  # Initialize an array to store the results
  M_np = np.zeros((2, 2, n_comp))

  # # compute the absolute value of H with external operator

  absH_check = AbsoluteValueH(Hu_prod, function_space = W2)
  absH_result = absH_check.assemble_N()

  # print the results

  # chechk function space of Hu

  print("WHAT DID I CREATE", absH_check)
  print()
  print(shape(absH_result))
  print()

  B_extra = Identity(2) + absH_result


  print(shape(B_extra))
  print()




  # Compute the square root of each 2x2 matrix

  for i in range(n_comp):

    # compute the |H(u)| + (H^2)^1/2

    Hu_matrix = np.array([[Hu00.dat.data[i], Hu01.dat.data[i]], [Hu10.dat.data[i], Hu11.dat.data[i]]])

    abs_H_i = sci.linalg.sqrtm(Hu_matrix)

    # compute the B matrix

    B_i = np.identity(2) + (1/alpha) * abs_H_i

    # compute the spectral radius of B_i

    eigs = sci.linalg.eigvals(B_i)
    spect_i = max(abs(eigs[0]), abs(eigs[1]))
    norm_B_i = spect_i**p2

    # multiply norm B by B

    prod_norm_B = norm_B_i * B_i

    # take determinant of B

    det_B_i = sci.linalg.det(B_i)**p1


    # create pointwise M

    M_i = det_B_i * prod_norm_B


    # now save this M

    M_np[:, :, i] = M_i




  # (10) Return M and its components

  M_func1 = Function(W2)
  M_func2 = Function(W2)
  M_func3 = Function(W2)
  M_func4 = Function(W2)
  M_func1.dat.data[:] = M_np[0, 0, :]
  M_func2.dat.data[:] = M_np[0, 1, :]
  M_func3.dat.data[:] = M_np[1, 0, :]
  M_func4.dat.data[:] = M_np[1, 1, :]
  M = as_matrix([ [ M_func1, M_func2 ] , [ M_func3, M_func4 ]] )




  return M, M_func1, M_func2, M_func3, M_func4

# Mesh Solver (Matrix Form w/o 1/J)

In [ ]:
# mesh solver


def mesh_solver(xi, eta, M_matrix, x, V1, deg = 1, rtol = 1e-8, stol = 1e-8, atol = 1e-50, maxit = 10, scale_BI = 1):

  # WINSLOW BASED ON (6.131) in HR

  # function to hold mesh solution
  mesh_sol = Function(V1)

  # Jacobian

  J_matrix = grad(mesh_sol)
  detJ = det(J_matrix)

  # JMJ part

  # The transformed nonlinear problem based on HR (3.8), (3.9) or (6.15), (6.131), (6.132, (6.133)
  JMJ_matrix = transpose(J_matrix) * M_matrix * J_matrix
  JMJ_inv = inv(JMJ_matrix)



  # create problem

  # test function

  v = TestFunction(V1)


  # WEAK FORM (based on Gockenbach and my own proofs)

  pt1 = inner(transpose(grad(v)), detJ * JMJ_inv) * dx

  # Boundary Integrals

  Minv = inv(M_matrix)


  if (scale_BI != 1) :
    print("I AM SCALING THE BI")
    print()


  BI1 = scale_BI * (- Minv[0, 1] * v[1] )*ds(1)
  BI2 = scale_BI * ( Minv[0, 1] * v[1] )*ds(2)
  BI3 = scale_BI * (- Minv[1, 0] * v[0] )*ds(3)
  BI4 = scale_BI * ( Minv[1, 0] * v[0] )*ds(4)

  BI = BI1 + BI2 + BI3 + BI4

  F = pt1 - BI


  # The initial guess (current (x,y) mesh)

  mesh_sol.interpolate(as_vector([x[0], x[1]]))


  # the Dirichlet bc's

  bc1 = DirichletBC(V1.sub(0), Constant(0), 1) # x=0 on left side
  bc2 = DirichletBC(V1.sub(0), Constant(1), 2) # x=1 on right side
  bc3 = DirichletBC(V1.sub(1), Constant(0), 3) # y=0 on bottom
  bc4 = DirichletBC(V1.sub(1), Constant(1), 4) # y=1 on top
  bcs = [bc1, bc2, bc3, bc4]



  # Solve, NewtonLS

  print("-"*125)
  print("Nonlinear Mesh Solve:")



  # solve(F == 0, mesh_sol, bcs = bcs, \
  #         solver_parameters = {'snes_converged_reason': None,\
  #                             'snes_monitor': None,\
  #                             "snes_stol": stol,\
  #                             "snes_rtol": rtol,\
  #                             "snes_atol": atol,\
  #                             "snes_max_it": maxit,\
  #                             "snes_linesearch_type": "l2",\
  #                              "snes_linesearch_monitor": None,})

  solve(F == 0, mesh_sol, bcs = bcs, \
        solver_parameters = {'snes_converged_reason': None,\
                            'snes_monitor': None,\
                            "snes_stol": stol,\
                            "snes_rtol": rtol,\
                            "snes_atol": atol,\
                            "snes_max_it": maxit} )
  print("-"*125)
  print()
  print()



  return mesh_sol


# MP-Iteration Code

In [ ]:
# MP - Iteration Code that uses differant mesh solve

def MP_iter_woJ(mesh, xi, eta, W1, V1, W2, V2, x, N, d, q, m, iter_count, epsilon, deg = 1, plot_phys = True, plot_mesh = True, rtol = 1e-8, stol = 1e-8, atol = 1e-50, maxit = 10, scale_BI = 1, plotsM = False):


  # set up for the loop
  # list to save physical erros

  phys_l2_errors = []
  phys_grad_errors = []


  # list to save progress

  result_list = []


  # set the loop for the set number of MP iterations

  for i in range(iter_count):


    # physical solve

    phys_sol, phys_error_1, phys_error_2, Jm, det_Jm = physical_solver(mesh, x, W2, epsilon, plots = plot_phys)


    # save errors

    phys_l2_errors.append(phys_error_1)
    phys_grad_errors.append(phys_error_2)


    # save x and phys sol to results

    result_list.append([x, phys_sol])


    # compute the mesh density function

    #M, M_func1, M_func2, M_func3, M_func4 = Sci_Hessian_M_3(mesh, x, phys_sol, Jm, det_Jm, W2, V2, d, q, m, epsilon, alpha_hat = 1)


    M, M_func1, M_func2, M_func3, M_func4  = ext_M(mesh, x, phys_sol, Jm, det_Jm, W2, V2,  d, q, m, epsilon, alpha_hat = 1)



    # print results for last epsilon


    check_eps = np.linalg.norm(epsilon - 0.03589489938459266)

    if check_eps < 1e-8: # print the plots



      # SHADOW MESH TEST

      # create shadow mesh

      shadow = UnitSquareMesh(N,N, quadrilateral = quadrilateral, diagonal = diagonal)
      #shadow = RectangleMesh(N, N, 0.25, 1, quadrilateral = quadrilateral, diagonal = diagonal)

      VOM = VertexOnlyMesh(mesh,vertexcoords=mesh.coordinates.dat.data, redundant=False)
      VOM_V = VectorFunctionSpace(VOM,"DG", 0)
      VOM_V_IO = VectorFunctionSpace(VOM.input_ordering,"DG", 0)

      test_VOM = Function(VOM_V).interpolate(final_x)
      test_VOM_IO = Function(VOM_V_IO).interpolate(test_VOM)

      shadow.coordinates.dat.data[:] = test_VOM_IO.dat.data[:]


      # now look at u on this mesh

      shadowV = FunctionSpace(shadow, "CG", 2)
      u_shadow = Function(shadowV)
      u_shadow.dat.data[:] = physical_sol.dat.data[:]

      # Lets put the exact solution on this mesh

      sol_known = Function(W2).interpolate(known_solution(x, epsilon))
      shadow_sol = Function(shadowV)
      shadow_sol.dat.data[:] = sol_known.dat.data[:]


      # # also put M on this mesh

      M_shadow1 = Function(shadowV)
      M_shadow1.dat.data[:] = M_func1.dat.data[:]

      M_shadow2 = Function(shadowV)
      M_shadow2.dat.data[:] = M_func2.dat.data[:]

      M_shadow3 = Function(shadowV)
      M_shadow3.dat.data[:] = M_func3.dat.data[:]

      M_shadow4 = Function(shadowV)
      M_shadow4.dat.data[:] = M_func4.dat.data[:]


      # plot the results
      fig, axes = plt.subplots(2, 2, figsize=(18, 18))

      u_shadow_array = u_shadow.dat.data
      shadow_sol_array = shadow_sol.dat.data
      sol_known_array = sol_known.dat.data
      physical_sol_array = physical_sol.dat.data


      M_shadow1_array = M_shadow1.dat.data
      M_func1_array = M_func1.dat.data
      M_shadow2_array = M_shadow2.dat.data
      M_func2_array = M_func2.dat.data
      M_func3_array = M_func3.dat.data
      M_func4_array = M_func4.dat.data
      M_shadow3_array = M_shadow3.dat.data
      M_shadow4_array = M_shadow4.dat.data


      # Calculate common min and max for the first two plots (for colour bar)
      vmin1 = min(np.min(u_shadow_array), np.min(physical_sol_array))
      vmax1 = max(np.max(u_shadow_array), np.max(physical_sol_array))
      # Calculate common min and max for the first two plots (for colour bar)
      vminKS = min(np.min(shadow_sol_array), np.min(sol_known_array))
      vmaxKS = max(np.max(shadow_sol_array), np.max(sol_known_array))



      # Calculate common min and max for the bottom two plots (for colour bar)
      vmin2 = min(np.min(M_shadow1_array), np.min(M_func1_array))
      vmax2 = max(np.max(M_shadow1_array), np.max(M_func1_array))
      # Calculate common min and max for the bottom two plots (for colour bar)
      vmin3 = min(np.min(M_shadow2_array), np.min(M_func2_array))
      vmax3 = max(np.max(M_shadow2_array), np.max(M_func2_array))
      # Calculate common min and max for the bottom two plots (for colour bar)
      vmin4 = min(np.min(M_shadow3_array), np.min(M_func3_array))
      vmax4 = max(np.max(M_shadow3_array), np.max(M_func3_array))
      # Calculate common min and max for the bottom two plots (for colour bar)
      vmin5 = min(np.min(M_shadow4_array), np.min(M_func4_array))
      vmax5 = max(np.max(M_shadow4_array), np.max(M_func4_array))


      # Plot the shadow mesh solution on the first subplot
      c_shad = tricontourf(u_shadow, axes=axes[0,0], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
      mp = triplot(shadow, axes = axes[0,0])
      axes[0,0].set_title('Shadow Mesh Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[0,0].set_xlabel(r"$x$")
      axes[0,0].set_ylabel(r"$y$")
      axes[0,0].set_aspect('equal')
      plt.colorbar(c_shad, ax=axes[0,0])
      # Plot the physical solution on the second subplot
      c = tricontourf(physical_sol, axes=axes[0,1], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
      mp = triplot(mesh, axes = axes[0,1])
      axes[0,1].set_title('Approx Physical Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[0,1].set_xlabel(r"$\xi$")
      axes[0,1].set_ylabel(r"$\eta$")
      axes[0,1].set_aspect('equal')
      plt.colorbar(c, ax=axes[0,1])

      # Plot the shadow mesh known solution
      c_shad = tricontourf(shadow_sol, axes=axes[1,0], cmap = custom_cmap, vmin = vminKS, vmax = vmaxKS)
      mp = triplot(shadow, axes = axes[1,0])
      axes[1,0].set_title('(Shadow) Known Physical Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[1,0].set_xlabel(r"$x$")
      axes[1,0].set_ylabel(r"$y$")
      axes[1,0].set_aspect('equal')
      plt.colorbar(c_shad, ax=axes[1,0])
      # Plot the known solution
      c = tricontourf(sol_known, axes=axes[1,1], cmap = custom_cmap, vmin = vminKS, vmax = vmaxKS)
      mp = triplot(mesh, axes = axes[1,1])
      axes[1,1].set_title('Known Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[1,1].set_xlabel(r"$\xi$")
      axes[1,1].set_ylabel(r"$\eta$")
      axes[1,1].set_aspect('equal')
      plt.colorbar(c, ax=axes[1,1])

      # Save the plots
      plt.tight_layout()
      plt.show()
      print()
      print()




      fig, axes = plt.subplots(4, 2, figsize=(24, 32))

      # Plot m00 on shadow mesh
      cm_shad = tricontourf(M_shadow1, axes=axes[0, 0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
      mp = triplot(shadow, axes = axes[0,0])
      axes[0, 0].set_title('m00 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[0, 0].set_xlabel(r"$x$")
      axes[0, 0].set_ylabel(r"$y$")
      #axes[2,0].set_aspect('equal')
      plt.colorbar(cm_shad, ax=axes[0,0])
      # Plot m00 on mesh
      cm = tricontourf(M_func1, axes=axes[0, 1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
      mp = triplot(mesh, axes = axes[0, 1])
      axes[0, 1].set_title('m00 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[0, 1].set_xlabel(r"$\xi$")
      axes[0, 1].set_ylabel(r"$\eta$")
      #axes[2,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[0, 1])


      # Plot m01 on shadow mesh
      cm_shad = tricontourf(M_shadow2, axes=axes[1,0], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
      mp = triplot(shadow, axes = axes[1,0])
      axes[1,0].set_title('m01 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[1,0].set_xlabel(r"$x$")
      axes[1,0].set_ylabel(r"$y$")
      #axes[3,0].set_aspect('equal')
      plt.colorbar(cm_shad, ax=axes[1,0])
      # Plot m01 on mesh
      cm = tricontourf(M_func2, axes=axes[1,1], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
      mp = triplot(mesh, axes = axes[1,1])
      axes[1,1].set_title('m01 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[1,1].set_xlabel(r"$\xi$")
      axes[1,1].set_ylabel(r"$\eta$")
      #axes[3,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[1,1])


      # Plot m10 on shadow mesh
      cm_shad = tricontourf(M_shadow3, axes=axes[2,0], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
      mp = triplot(shadow, axes = axes[2,0])
      axes[2,0].set_title('m10 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[2,0].set_xlabel(r"$x$")
      axes[2,0].set_ylabel(r"$y$")
      #axes[3,0].set_aspect('equal')
      plt.colorbar(cm_shad, ax=axes[2,0])
      # Plot m10 on mesh
      cm = tricontourf(M_func3, axes=axes[2,1], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
      mp = triplot(mesh, axes = axes[2,1])
      axes[2,1].set_title('m10 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[2,1].set_xlabel(r"$\xi$")
      axes[2,1].set_ylabel(r"$\eta$")
      #axes[3,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[2,1])


      # Plot m11 on shadow mesh
      cm_shad = tricontourf(M_shadow4, axes=axes[3,0], cmap = custom_cmap, vmin = vmin5, vmax = vmax5)
      mp = triplot(shadow, axes = axes[3,0])
      axes[3,0].set_title('m11 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[3,0].set_xlabel(r"$x$")
      axes[3,0].set_ylabel(r"$y$")
      #axes[3,0].set_aspect('equal')
      plt.colorbar(cm_shad, ax=axes[3,0])
      # Plot m11 on mesh
      cm = tricontourf(M_func4, axes=axes[3,1], cmap = custom_cmap, vmin = vmin5, vmax = vmax5)
      mp = triplot(mesh, axes = axes[3,1])
      axes[3,1].set_title('m11 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
      axes[3,1].set_xlabel(r"$\xi$")
      axes[3,1].set_ylabel(r"$\eta$")
      #axes[3,1].set_aspect('equal')
      plt.colorbar(cm, ax=axes[3,1])

      # Save the plots
      plt.tight_layout()
      plt.show()
      print()
      print()
      print()
      print()









    print("M NORMS")
    print("_"*100)
    print()
    print('Norm of the M_00 is', norm(sqrt(det_Jm) * M[0, 0]))
    print('Norm of the M_01 is', norm(sqrt(det_Jm) * M[0, 1]))
    print('Norm of the M_10 is', norm(sqrt(det_Jm) * M[1, 0]))
    print('Norm of the M_11 is', norm(sqrt(det_Jm) * M[1, 1]))
    print("_"*100)
    print()
    print()



    # Mesh Solve

    mesh_sol = mesh_solver(xi, eta, M, x, V1, deg = deg, rtol = rtol, stol = stol, atol = atol, maxit = maxit, scale_BI = scale_BI)


    # analyize mesh convergence
    # if i > 0:
    #   mesh_check = norm(mesh_sol - x)


    # UPDATE x

    x = mesh_sol


  # do a final physical solve on the last mesh

  phys_sol, phys_error_1, phys_error_2, Jm, det_Jm = physical_solver(mesh, x, W2, epsilon, plots = plot_phys)
  phys_l2_errors.append(phys_error_1)
  phys_grad_errors.append(phys_error_2)


  # save x and phys sol to results

  result_list.append([x, phys_sol])








  return result_list, x, phys_l2_errors, phys_grad_errors, phys_sol, M, M_func1, M_func2, M_func3, M_func4

# The Test

In [ ]:

# Set My Parameters

epsilon = 0.5 # start at 0.2 if uniform!
N = 24
diagonal = "right"
quadrilateral = False
x_choice = 1 # 0 = identity function, 1 = map uniform to shishkin, 2 = map shishkin to uniform
xi_choice = 0 # 0 = uniform mesh, 1 = shishkin mesh
custom_cmap = "GnBu"
deg = 2
MPIter = 3
cont_steps = 25
# SNES stopping criteria
stol = 1e-8
rtol = 1e-8
atol = 1e-50
maxit = 20
# for Mesh Density Function
d = 2
q = 2
m = 1


# save all progress
all_results = []

# scaling the BI's?
scale_BI = 1


# list to save the x's
x_list = []




# Set xi


if xi_choice == 0: # ci is uniform

  mesh = UnitSquareMesh(N, N, quadrilateral = quadrilateral, diagonal = diagonal)
  xi, eta = SpatialCoordinate(mesh)


if xi_choice == 1: # xi is shishkin

  epsm = 0.025
  beta = 0.99/np.sqrt(2)
  tau_a = 1/2
  tau_b = 2*epsm*(1/beta)*np.log(N)
  tau = min(tau_a, tau_b)

  hB = 2*tau / N
  hI = 2*(1-tau) / N


  part1 = np.arange(0, tau, hB)
  part2 = np.arange(tau, 1.0+hI, hI)

  # w_x = np.concatenate((part1, part2[:-1]))
  # w_y = np.concatenate((part1, part2[:-1]))
  w_x = np.concatenate((part1, part2))
  w_y = np.concatenate((part1, part2))

  mesh = TensorRectangleMesh(w_x, w_y, quadrilateral = quadrilateral, diagonal = diagonal)
  xi, eta = SpatialCoordinate(mesh)




# function spaces

W2 = FunctionSpace(mesh, "CG", deg)
V2 = VectorFunctionSpace(mesh, "CG", deg)
W1 = FunctionSpace(mesh, "CG", 1)
V1 = VectorFunctionSpace(mesh, "CG", 1)


# set x


if x_choice == 0: # x is identity function

  x = Function(V1).interpolate(as_vector([xi, eta]))

elif x_choice == 1: # x maps uniform to shihskin

  epsx = 0.025
  beta = 0.99/np.sqrt(2)
  tau_a = 1/2
  tau_b = 2*epsx*(1/beta)*np.log(N)
  tau = min(tau_a, tau_b)

  x_shish = conditional( xi < 0.5, 2*xi*tau, tau + 2*(1-tau)*(xi-1/2) )
  y_shish = conditional( eta < 0.5, 2*eta*tau, tau + 2*(1-tau)*(eta-1/2) )

  x = Function(V1).interpolate(as_vector([x_shish, y_shish]))


elif x_choice == 2: # x maps shihskin to uniform

  x_shish = conditional( xi <= tau, (1/(2*tau))*xi, (1/(2*(1-tau)))*( xi + 1 - 2*tau) )
  y_shish = conditional( eta < tau, (1/(2*tau))*eta, (1/(2*(1-tau)))*( eta + 1 - 2*tau))

  x = Function(V1).interpolate(as_vector([x_shish, y_shish]))


# evaluate our starting x function so that we can visualize

# solution data
old_mesh_vals = mesh.coordinates.dat.data
xsol = np.array(x.at(old_mesh_vals))[:,0]
ysol = np.array(x.at(old_mesh_vals))[:,1]

# create mesh
initial_phys_mesh = UnitSquareMesh(N, N, diagonal  = diagonal, quadrilateral = quadrilateral)
nx = len(xsol)
new_mesh_vals = np.zeros((nx, 2))
new_mesh_vals[:,0] = xsol
new_mesh_vals[:,1] = ysol
initial_phys_mesh.coordinates.dat.data[:] = new_mesh_vals


print()

#physical error list

physical_l2_e = []
physical_grad_e = []
figures = []
fig_for_later = []








# Set the epsilon continuation + MP solves




for i in range(cont_steps):


  physical_l2_e.append(epsilon)
  physical_grad_e.append(epsilon)


  print("_"*60 + " epsilon = " + str(epsilon) + " " + "_"*60 )
  print()
  print()
  print()




  plotsM = False


  # MP Iteration

  results, final_x, phys_e, phys_e2, physical_sol, M_final, M_func1, M_func2, M_func3, M_func4 = MP_iter_woJ(mesh, xi, eta, W1, V1, W2, V2, x, N, d, q, m, MPIter, epsilon,\
                                                                          plot_phys = False, plot_mesh = False,\
                                                                          rtol = rtol, stol = stol, atol = atol, maxit = maxit, plotsM = plotsM)


  # save progress

  # all_results.append([epsilon, results])


  # save the x
  x_list.append(final_x)

  # save errors from physical problem
  physical_l2_e.append(phys_e)
  physical_grad_e.append(phys_e2)


  # SHADOW MESH TEST

  # create shadow mesh

  shadow = UnitSquareMesh(N,N, quadrilateral = quadrilateral, diagonal = diagonal)
  #shadow = RectangleMesh(N, N, 0.25, 1, quadrilateral = quadrilateral, diagonal = diagonal)

  VOM = VertexOnlyMesh(mesh,vertexcoords=mesh.coordinates.dat.data, redundant=False)
  VOM_V = VectorFunctionSpace(VOM,"DG", 0)
  VOM_V_IO = VectorFunctionSpace(VOM.input_ordering,"DG", 0)

  test_VOM = Function(VOM_V).interpolate(final_x)
  test_VOM_IO = Function(VOM_V_IO).interpolate(test_VOM)

  shadow.coordinates.dat.data[:] = test_VOM_IO.dat.data[:]


  # now look at u on this mesh

  shadowV = FunctionSpace(shadow, "CG", 2)
  u_shadow = Function(shadowV)
  u_shadow.dat.data[:] = physical_sol.dat.data[:]

  # Lets put the exact solution on this mesh

  sol_known = Function(W2).interpolate(known_solution(final_x, epsilon))
  shadow_sol = Function(shadowV)
  shadow_sol.dat.data[:] = sol_known.dat.data[:]


  # # also put M on this mesh

  M_shadow1 = Function(shadowV)
  M_shadow1.dat.data[:] = M_func1.dat.data[:]

  M_shadow2 = Function(shadowV)
  M_shadow2.dat.data[:] = M_func2.dat.data[:]

  M_shadow3 = Function(shadowV)
  M_shadow3.dat.data[:] = M_func3.dat.data[:]

  M_shadow4 = Function(shadowV)
  M_shadow4.dat.data[:] = M_func4.dat.data[:]


  # plot the results
  fig, axes = plt.subplots(2, 2, figsize=(18, 18))

  u_shadow_array = u_shadow.dat.data
  shadow_sol_array = shadow_sol.dat.data
  sol_known_array = sol_known.dat.data
  physical_sol_array = physical_sol.dat.data


  # Interpolate mesh density functions onto a rectangle mesh

  rect_mesh = RectangleMesh(int(N/4), N, 0.25, 1)
  rectV = FunctionSpace(rect_mesh, "CG", 2)
  M_func1_rect = Function(rectV).interpolate(M_func1)
  M_func2_rect = Function(rectV).interpolate(M_func2)
  M_func3_rect = Function(rectV).interpolate(M_func3)
  M_func4_rect = Function(rectV).interpolate(M_func4)


  # small shadow mesh

  small_shadow = RectangleMesh(int(N/2), N, 0.25, 1)
  small_shad_V = FunctionSpace(small_shadow, "CG", 2)
  Mshad1_rect = Function(small_shad_V).interpolate(M_shadow1)
  Mshad2_rect = Function(small_shad_V).interpolate(M_shadow2)
  Mshad3_rect = Function(small_shad_V).interpolate(M_shadow3)
  Mshad4_rect = Function(small_shad_V).interpolate(M_shadow4)


  M_shadow1_array = M_shadow1.dat.data
  M_func1_array = M_func1.dat.data
  M_shadow2_array = M_shadow2.dat.data
  M_func2_array = M_func2.dat.data
  M_func3_array = M_func3.dat.data
  M_func4_array = M_func4.dat.data
  M_shadow3_array = M_shadow3.dat.data
  M_shadow4_array = M_shadow4.dat.data


  # Calculate common min and max for the first two plots (for colour bar)
  vmin1 = min(np.min(u_shadow_array), np.min(physical_sol_array))
  vmax1 = max(np.max(u_shadow_array), np.max(physical_sol_array))
  # Calculate common min and max for the first two plots (for colour bar)
  vminKS = min(np.min(shadow_sol_array), np.min(sol_known_array))
  vmaxKS = max(np.max(shadow_sol_array), np.max(sol_known_array))



  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin2 = min(np.min(M_shadow1_array), np.min(M_func1_array))
  vmax2 = max(np.max(M_shadow1_array), np.max(M_func1_array))
  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin3 = min(np.min(M_shadow2_array), np.min(M_func2_array))
  vmax3 = max(np.max(M_shadow2_array), np.max(M_func2_array))
  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin4 = min(np.min(M_shadow3_array), np.min(M_func3_array))
  vmax4 = max(np.max(M_shadow3_array), np.max(M_func3_array))
  # Calculate common min and max for the bottom two plots (for colour bar)
  vmin5 = min(np.min(M_shadow4_array), np.min(M_func4_array))
  vmax5 = max(np.max(M_shadow4_array), np.max(M_func4_array))


  # Plot the shadow mesh solution on the first subplot
  c_shad = tricontourf(u_shadow, axes=axes[0,0], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
  mp = triplot(shadow, axes = axes[0,0])
  axes[0,0].set_title('Shadow Mesh Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0,0].set_xlabel(r"$x$")
  axes[0,0].set_ylabel(r"$y$")
  axes[0,0].set_aspect('equal')
  plt.colorbar(c_shad, ax=axes[0,0])
  # Plot the physical solution on the second subplot
  c = tricontourf(physical_sol, axes=axes[0,1], cmap = custom_cmap, vmin = vmin1, vmax = vmax1)
  mp = triplot(mesh, axes = axes[0,1])
  axes[0,1].set_title('Approx Physical Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0,1].set_xlabel(r"$\xi$")
  axes[0,1].set_ylabel(r"$\eta$")
  axes[0,1].set_aspect('equal')
  plt.colorbar(c, ax=axes[0,1])

  # Plot the shadow mesh known solution
  c_shad = tricontourf(shadow_sol, axes=axes[1,0], cmap = custom_cmap, vmin = vminKS, vmax = vmaxKS)
  mp = triplot(shadow, axes = axes[1,0])
  axes[1,0].set_title('(Shadow) Known Physical Solution for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,0].set_xlabel(r"$x$")
  axes[1,0].set_ylabel(r"$y$")
  axes[1,0].set_aspect('equal')
  plt.colorbar(c_shad, ax=axes[1,0])
  # Plot the known solution
  c = tricontourf(sol_known, axes=axes[1,1], cmap = custom_cmap, vmin = vminKS, vmax = vmaxKS)
  mp = triplot(mesh, axes = axes[1,1])
  axes[1,1].set_title('Known Solution at ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,1].set_xlabel(r"$\xi$")
  axes[1,1].set_ylabel(r"$\eta$")
  axes[1,1].set_aspect('equal')
  plt.colorbar(c, ax=axes[1,1])

  # Save the plots
  plt.tight_layout()
  plt.close(fig)
  figures.append(fig)


  # just take the m00 plots

  fig, axes = plt.subplots(1, 2, figsize=(14, 6))

  # Plot m0 on shadow mesh
  cm_shad = tricontourf(M_shadow1, axes=axes[0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  mp = triplot(shadow, axes = axes[0])
  axes[0].set_title('m00 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0].set_xlabel(r"$x$")
  axes[0].set_ylabel(r"$y$")
  #axes[2,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[0])
  # Plot m0 on mesh
  cm = tricontourf(M_func1, axes=axes[1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  mp = triplot(mesh, axes = axes[1])
  axes[1].set_title('m00 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1].set_xlabel(r"$\xi$")
  axes[1].set_ylabel(r"$\eta$")
  #axes[2,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[1])

  # save the plots
  plt.tight_layout()
  plt.close(fig)
  fig_for_later.append(fig)



  fig, axes = plt.subplots(4, 2, figsize=(24, 32))

  # Plot m00 on shadow mesh
  cm_shad = tricontourf(M_shadow1, axes=axes[0, 0], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  mp = triplot(shadow, axes = axes[0,0])
  axes[0, 0].set_title('m00 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0, 0].set_xlabel(r"$x$")
  axes[0, 0].set_ylabel(r"$y$")
  #axes[2,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[0,0])
  # Plot m00 on mesh
  cm = tricontourf(M_func1, axes=axes[0, 1], cmap = custom_cmap, vmin = vmin2, vmax = vmax2)
  mp = triplot(mesh, axes = axes[0, 1])
  axes[0, 1].set_title('m00 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[0, 1].set_xlabel(r"$\xi$")
  axes[0, 1].set_ylabel(r"$\eta$")
  #axes[2,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[0, 1])


  # Plot m01 on shadow mesh
  cm_shad = tricontourf(M_shadow2, axes=axes[1,0], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
  mp = triplot(shadow, axes = axes[1,0])
  axes[1,0].set_title('m01 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,0].set_xlabel(r"$x$")
  axes[1,0].set_ylabel(r"$y$")
  #axes[3,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[1,0])
  # Plot m01 on mesh
  cm = tricontourf(M_func2, axes=axes[1,1], cmap = custom_cmap, vmin = vmin3, vmax = vmax3)
  mp = triplot(mesh, axes = axes[1,1])
  axes[1,1].set_title('m01 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[1,1].set_xlabel(r"$\xi$")
  axes[1,1].set_ylabel(r"$\eta$")
  #axes[3,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[1,1])


  # Plot m10 on shadow mesh
  cm_shad = tricontourf(M_shadow3, axes=axes[2,0], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
  mp = triplot(shadow, axes = axes[2,0])
  axes[2,0].set_title('m10 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[2,0].set_xlabel(r"$x$")
  axes[2,0].set_ylabel(r"$y$")
  #axes[3,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[2,0])
  # Plot m10 on mesh
  cm = tricontourf(M_func3, axes=axes[2,1], cmap = custom_cmap, vmin = vmin4, vmax = vmax4)
  mp = triplot(mesh, axes = axes[2,1])
  axes[2,1].set_title('m10 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[2,1].set_xlabel(r"$\xi$")
  axes[2,1].set_ylabel(r"$\eta$")
  #axes[3,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[2,1])


  # Plot m11 on shadow mesh
  cm_shad = tricontourf(M_shadow4, axes=axes[3,0], cmap = custom_cmap, vmin = vmin5, vmax = vmax5)
  mp = triplot(shadow, axes = axes[3,0])
  axes[3,0].set_title('m11 on Shadow Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[3,0].set_xlabel(r"$x$")
  axes[3,0].set_ylabel(r"$y$")
  #axes[3,0].set_aspect('equal')
  plt.colorbar(cm_shad, ax=axes[3,0])
  # Plot m11 on mesh
  cm = tricontourf(M_func4, axes=axes[3,1], cmap = custom_cmap, vmin = vmin5, vmax = vmax5)
  mp = triplot(mesh, axes = axes[3,1])
  axes[3,1].set_title('m11 on Mesh for ' + r'$\epsilon$' + ' = ' + "%.8f"%epsilon)
  axes[3,1].set_xlabel(r"$\xi$")
  axes[3,1].set_ylabel(r"$\eta$")
  #axes[3,1].set_aspect('equal')
  plt.colorbar(cm, ax=axes[3,1])

  # Save the plots
  plt.tight_layout()
  plt.close(fig)
  figures.append(fig)


  check_eps = np.linalg.norm(epsilon - 0.03589489938459266)

  if check_eps < 1e-8: # print the plots


    display(figures[-2])
    print()
    display(fig_for_later[-1])
    print()
    print()
    print()
    print()




  # save the results for this epsilon to vtk file

  # save both of these to a VTK File
  file_path = '/content/drive/My Drive/Madden_plots/Hessian_Eps_Cont_Tests/July24_Solns_FirstTests_at_eps_' + str(epsilon) + '.pvd'
  outfile = VTKFile(file_path)
  outfile.write(u_shadow, time = 1) #approx on shadow mesh
  u_shadow.dat.data[:] = shadow_sol.dat.data[:] # need to replace approx soln with known solution to put in the file
  outfile.write(u_shadow, time = 2) # known on shadow mesh


  # do the same for the m00, m01, m10, m11
  file_path = '/content/drive/My Drive/Madden_plots/Hessian_Eps_Cont_Tests/July24_MComps_FirstTests_at_eps_' + str(epsilon) + '.pvd'
  outfile = VTKFile(file_path)
  outfile.write(M_shadow1, time = 1)
  M_shadow1.dat.data[:] = M_shadow2.dat.data[:]
  outfile.write(M_shadow1, time = 2)
  M_shadow1.dat.data[:] = M_shadow3.dat.data[:]
  outfile.write(M_shadow1, time = 3)
  M_shadow1.dat.data[:] = M_shadow4.dat.data[:]
  outfile.write(M_shadow1, time = 4)


  # construct x as the new initial guess
  # update epsilon


  epsilon = epsilon*(9/10)


  x = final_x









____________________________________________________________ epsilon = 0.5 ____________________________________________________________





KeyboardInterrupt: 

In [ ]:
print( type( (Function(W2).interpolate(M_final[0, 0]).dat.data)[0] ))

In [ ]:
# display all my figures

i = 1
while i <= len(figures):
  display(figures[i-1]) # physical solution
  display(figures[i]) # components of M
  i = i+ 2
  print()
  print()
  print()
  print()
  print()


# figure_i = 0
# for i in range(10):

#   display(figures[figure_i])
#   figure_i = figure_i + 5
#   print()
#   print()
#   print()
#   print()

# display(figures[-1])

In [ ]:
# # print norms of these x's

# for i in range(len(x_list)):
#   print(norm(x_list[i] - x_list_scaled[i]))
#   print()